In [23]:
!bash source ~/opt/ros/melodic/setup.bash

bash: source: No such file or directory


In [1]:
from canlib import canlib, kvadblib
import threading
import rospy


_dbName_ = "cankingDB_ioniq_dgist_mod5.dbc"
db = kvadblib.Dbc(filename=_dbName_)


ModuleNotFoundError: No module named 'rospkg'

In [18]:
controlCMD = db.get_message_by_name('Control_CMD')
drivingCMD = db.get_message_by_name('Driving_CMD')

cmsg = controlCMD.bind()
dmsg = drivingCMD.bind()

__CH_NUM__ = 0

try:
    ch = canlib.openChannel(__CH_NUM__, canlib.canOPEN_ACCEPT_VIRTUAL)
    ch.setBusOutputControl(canlib.canDRIVER_NORMAL)
    ch.setBusParams(__BITRATE__)
    ch.busOn()
except Exception as ex:
    print("Error occured", ex)
    
    
    

Error occured Specified device not found (-3)


In [ ]:
class sender():
    def __init__(self, ch, db):
        self.ch = ch
        controlCMD = db.get_message_by_name('Control_CMD')
        drivingCMD = db.get_message_by_name('Driving_CMD')
        
        self.cmsg = controlCMD.bind()
        self.dmsg = drivingCMD.bind()
        
        print("Init Connection... send Alive CMD, Check Interface!")
        heartbeat = threading.Thread(target = AliveSender)
        
    def AliveSender(self):
        aliveVar = 0
        while True:
            self.cmsg.AliveCount.phys = 255 % aliveVar
            self.ch.write(self.cmsg)
            aliveVar += 1    
            
    def setAngularSpeed(self, value):
        value = value if value < 255 else 255
        self.cmsg.AngulaSpeedCMD.phys = value
        self.ch.write(self.cmsg)
        
    def setAccelCMD(self, value):
        __MAX__ = 700
        '''TODO: Retrieve MAX Value at rosparam'''
        value = value if value < __MAX__ else __MAX__
        self.dmsg.AccelCMD.phys = value 
        self.ch.write(self.dmsg)
    
    def setBrakeCMD(self, value):
        __MAX__ = 27000
        value = value if value < __MAX__ else __MAX__
        self.dmsg.BrakeCMD.phys = value 
        self.ch.write(self.dmsg)
            
    def setSteerCMD(self, value):
        __MAX__ = 570
        value = value if abs(value) < __MAX__ else __MAX__
        self.dmsg.SteerCMD.phys = value 
        self.ch.write(self.dmsg)
        
    def setGearCMD(self, value):
        cmd = [0, 5 ,6 ,7]
        if not value in cmd:
            #Rogue Data
            return -1
        self.dmsg.GearShiftCMD.phys = value
        self.ch.write(self.dmsg)
        
